# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [15]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

We create two python functions for determining the facets to apply boundary conditions to

In [16]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, we create a  marker based on these two functions

In [17]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [18]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [19]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [20]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [21]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [22]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [23]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [24]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [25]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [26]:
problem = NonlinearProblem(F, u, bcs)

and then create and customize the Newton solver

In [27]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


We create a function to plot the solution at each time step.

In [28]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

Finally, we solve the problem over several time steps, updating the z-component of the traction

In [29]:
import logging
from dolfinx import *  # Assuming the required dolfinx modules are imported
from matplotlib import pyplot as plt
import pyvista as pv

log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(4961, len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()


2024-07-27 09:34:58.581 (  33.397s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:03.659 (  38.475s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:06.814 (  41.630s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2024-07-27 09:35:07.162 (  41.979s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:09.705 (  44.521s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 2.43261 (tol = 1e-08) r (rel) = 0.0146837(tol = 1e-08)
2024-07-27 09:35:10.030 (  44.846s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:12.475 (  47.291s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


2024-07-27 09:35:23.896 (  58.712s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 8: r (abs) = 2.65767e-11 (tol = 1e-08) r (rel) = 1.60422e-13(tol = 1e-08)
2024-07-27 09:35:23.896 (  58.712s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 8 iterations and 8 linear solver iterations.
2024-07-27 09:35:24.539 (  59.356s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:27.307 (  62.123s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:29.783 (  64.599s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2024-07-27 09:35:30.120 (  64.936s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:32.601 (  67.417s) [main            ]       NewtonSolver.cp

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


2024-07-27 09:35:49.383 (  84.199s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:51.866 (  86.682s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:54.231 (  89.047s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2024-07-27 09:35:54.545 (  89.361s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:56.743 (  91.559s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 5.33026 (tol = 1e-08) r (rel) = 0.0472992(tol = 1e-08)
2024-07-27 09:35:57.046 (  91.863s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:35:59.207 (  94.023s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration

...

Time step 7, Number of iterations 6, Load [  0.    0.  -10.5]


2024-07-27 09:37:36.385 ( 191.201s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:37:38.456 ( 193.272s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:37:40.239 ( 195.055s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1.06336 (tol = 1e-08) r (rel) = 0.031085(tol = 1e-08)
2024-07-27 09:37:40.526 ( 195.342s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:37:42.386 ( 197.202s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 2.0477 (tol = 1e-08) r (rel) = 0.0598598(tol = 1e-08)
2024-07-27 09:37:42.692 ( 197.508s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:37:44.558 ( 199.374s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 4

Time step 8, Number of iterations 6, Load [  0.   0. -12.]


2024-07-27 09:37:49.059 ( 203.875s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:37:51.396 ( 206.212s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:37:53.496 ( 208.312s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 0.898789 (tol = 1e-08) r (rel) = 0.0309666(tol = 1e-08)
2024-07-27 09:37:53.825 ( 208.641s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:37:55.958 ( 210.775s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 1.38354 (tol = 1e-08) r (rel) = 0.0476679(tol = 1e-08)
2024-07-27 09:37:56.255 ( 211.071s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 09:37:58.066 ( 212.882s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteratio

Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


<img src="./deformation.gif" alt="gif" class="bg-primary mb-1" width="800px">